In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import tqdm
import joblib
import utils as utils
from sklearn.linear_model import LinearRegression

base_dir = os.path.join(os.getcwd(), '..')
sys.path.append(base_dir)

import fit_spatial_GP as plaingp
import fit_spatial_FaIRGP as fairgp
from src.models.utils_spatial import compute_means
from src.evaluation.metrics import spearman_correlation

<IPython.core.display.Javascript object>

In [3]:
def weighted_mean(x, wlat):
    mu = torch.sum(x * wlat, dim=(1, 2)) / (x.size(2) * wlat.sum())
    return mu

def compute_deterministic_metrics(prediction, groundtruth, wlat):
    # Compute raw distances metrics
    difference = prediction.sub(groundtruth)
    mean_bias = weighted_mean(difference, wlat).mean()
    rmse = weighted_mean(torch.square(difference), wlat).mean().sqrt()
    mae = weighted_mean(torch.abs(difference), wlat).mean()

    # Compute spearman correlation
    corr = spearman_correlation(prediction.flatten(), groundtruth.flatten())

    # Encapsulate results in output dictionnary
    output = {'mb': mean_bias.item(),
              'rmse': rmse.item(),
              'mae': mae.item(),
              'corr': corr}
    return output

def compute_probabilistic_metrics(prediction, groudtruth, wlat):
    ll = weighted_mean(prediction.log_prob(groundtruth), wlat).mean()
    lb, ub = prediction.icdf(torch.tensor(0.025)), prediction.icdf(torch.tensor(0.975))
    mask = (groundtruth >= lb) & (groundtruth <= ub)
    calib95 = weighted_mean(mask.float(), wlat).mean()
    
    mu, sigma = prediction.mean, prediction.stddev
    y = (groundtruth - mu) / sigma
    norm = torch.distributions.Normal(0, 1)
    crps = sigma * (y * (2 * norm.cdf(y) - 1) + 2 * norm.log_prob(y).exp() - 1 / np.sqrt(np.pi))
    crps = weighted_mean(crps, wlat).mean()
    
    output = {'ll': ll.item(),
              'calib95': calib95.item(),
              'CRPS': crps.item()}
    return output

In [4]:
ssps = ['SSP126', 'SSP245', 'SSP370', 'SSP585']

## FaIR pattern scaling

### SSP126

In [5]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp245', 'ssp370', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126']}}
test_data = plaingp.make_data(test_cfg)

In [6]:
scenarios = train_data.scenarios
all_tas = scenarios.tas
wlat = torch.cos(torch.deg2rad(scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]
glob_tas = weighted_mean(all_tas, wlat)

pattern_scaling = LinearRegression()
pattern_scaling.fit(glob_tas[:, None], all_tas.reshape(all_tas.size(0), -1))

LinearRegression()

In [7]:
time_slice = slice(-21, None)

pred = compute_means(test_data.scenarios, pattern_scaling)['ssp126']
metrics_ssp126 = compute_deterministic_metrics(pred[time_slice],
                                               test_data.scenarios[0].tas[time_slice],
                                               wlat)

### SSP245

In [8]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp370', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp245']}}
test_data = plaingp.make_data(test_cfg)

In [9]:
scenarios = train_data.scenarios
all_tas = scenarios.tas
wlat = torch.cos(torch.deg2rad(scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]
glob_tas = weighted_mean(all_tas, wlat)

pattern_scaling = LinearRegression()
pattern_scaling.fit(glob_tas[:, None], all_tas.reshape(all_tas.size(0), -1))

LinearRegression()

In [10]:
time_slice = slice(-21, None)

pred = compute_means(test_data.scenarios, pattern_scaling)['ssp245']
metrics_ssp245 = compute_deterministic_metrics(pred[time_slice],
                                               test_data.scenarios[0].tas[time_slice],
                                               wlat)

### SSP370

In [11]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp370']}}
test_data = plaingp.make_data(test_cfg)

In [12]:
scenarios = train_data.scenarios
all_tas = scenarios.tas
wlat = torch.cos(torch.deg2rad(scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]
glob_tas = weighted_mean(all_tas, wlat)

pattern_scaling = LinearRegression()
pattern_scaling.fit(glob_tas[:, None], all_tas.reshape(all_tas.size(0), -1))

LinearRegression()

In [13]:
time_slice = slice(-21, None)

pred = compute_means(test_data.scenarios, pattern_scaling)['ssp370']
metrics_ssp370 = compute_deterministic_metrics(pred[time_slice],
                                               test_data.scenarios[0].tas[time_slice],
                                               wlat)

### SSP585

In [14]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp370']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp585']}}
test_data = plaingp.make_data(test_cfg)

In [15]:
scenarios = train_data.scenarios
all_tas = scenarios.tas
wlat = torch.cos(torch.deg2rad(scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]
glob_tas = weighted_mean(all_tas, wlat)

pattern_scaling = LinearRegression()
pattern_scaling.fit(glob_tas[:, None], all_tas.reshape(all_tas.size(0), -1))

LinearRegression()

In [16]:
time_slice = slice(-21, None)

pred = compute_means(test_data.scenarios, pattern_scaling)['ssp585']
metrics_ssp585 = compute_deterministic_metrics(pred[time_slice],
                                               test_data.scenarios[0].tas[time_slice],
                                               wlat)

In [18]:
metrics_pattern_scaling = [metrics_ssp126, metrics_ssp245, metrics_ssp370, metrics_ssp585]
scores_pattern_scaling = pd.DataFrame(metrics_pattern_scaling)
scores_pattern_scaling.index = ssps
scores_pattern_scaling.index.name = 'SSP'

In [19]:
test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126', 'ssp245', 'ssp370', 'ssp585']}}
test_data = plaingp.make_data(test_cfg)

pattern_scaling = joblib.load('../data/pattern_scaling_model.joblib')
wlat = torch.cos(torch.deg2rad(test_data.scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]

In [20]:
pattern_scaled = compute_means(test_data.scenarios, pattern_scaling)

In [21]:
time_slice = slice(-21, None)

metrics_pattern_scaling = []
for name, preds in pattern_scaled.items():
    
    metrics_scenario = compute_deterministic_metrics(preds[time_slice],
                                                     test_data.scenarios[name].tas[time_slice],
                                                     wlat)
    metrics_pattern_scaling.append(metrics_scenario)

In [22]:
metrics_pattern_scaling = [metrics_ssp126, metrics_ssp245, metrics_ssp370, metrics_ssp585]
scores_pattern_scaling = pd.DataFrame(metrics_pattern_scaling)
scores_pattern_scaling.index = ssps
scores_pattern_scaling.index.name = 'SSP'

In [23]:
scores_pattern_scaling

,mb,rmse,mae,corr
SSP,,,,
SSP126,0.017865,0.673905,0.485556,0.775864
SSP245,-0.048413,0.371226,0.272364,0.966141
SSP370,-0.100107,0.662458,0.483243,0.944825
SSP585,-0.409348,0.810060,0.596555,0.966578


## PlainGP

### SSP126

In [24]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp245', 'ssp370', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126']}}
test_data = plaingp.make_data(test_cfg)

model = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp126/SpatialPlainGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [25]:
test_scenarios = test_data.scenarios
Xtest = test_scenarios.glob_inputs[:, 1:]
Xtest = (Xtest - model.mu) / model.sigma
test_tas = test_scenarios.tas
model = model.eval()

In [26]:
with torch.no_grad():
    posterior = model.posterior(Xtest, diag=False)
    posterior = model.likelihood(posterior)

In [27]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
ntimes = len(test_scenarios[0].timesteps)
time_slice = slice(-21, None)

prior_mean = model.mu_targets.view(1, nlat, nlon).repeat(ntimes, 1, 1)
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction =  model.sigma_targets.view(1, nlat, nlon) * posterior_mean
posterior_mean = prior_mean + posterior_mean

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets.view(1, nlat, nlon) * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]
wlat = torch.cos(torch.deg2rad(test_scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]

In [28]:
metrics_ssp126 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP245

In [29]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp370', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp245']}}
test_data = plaingp.make_data(test_cfg)

model = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp245/SpatialPlainGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [30]:
test_scenarios = test_data.scenarios
Xtest = test_scenarios.glob_inputs[:, 1:]
Xtest = (Xtest - model.mu) / model.sigma
test_tas = test_scenarios.tas
model = model.eval()

In [31]:
with torch.no_grad():
    posterior = model.posterior(Xtest, diag=False)
    posterior = model.likelihood(posterior)

In [32]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
ntimes = len(test_scenarios[0].timesteps)
time_slice = slice(-21, None)

prior_mean = model.mu_targets.view(1, nlat, nlon).repeat(ntimes, 1, 1)
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction =  model.sigma_targets.view(1, nlat, nlon) * posterior_mean
posterior_mean = prior_mean + posterior_mean

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets.view(1, nlat, nlon) * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]
wlat = torch.cos(torch.deg2rad(test_scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]

In [33]:
metrics_ssp245 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP370

In [34]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp370']}}
test_data = plaingp.make_data(test_cfg)

model = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp370/SpatialPlainGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [35]:
test_scenarios = test_data.scenarios
Xtest = test_scenarios.glob_inputs[:, 1:]
Xtest = (Xtest - model.mu) / model.sigma
test_tas = test_scenarios.tas
model = model.eval()

In [36]:
with torch.no_grad():
    posterior = model.posterior(Xtest, diag=False)
    posterior = model.likelihood(posterior)

In [37]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
ntimes = len(test_scenarios[0].timesteps)
time_slice = slice(-21, None)

prior_mean = model.mu_targets.view(1, nlat, nlon).repeat(ntimes, 1, 1)
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction =  model.sigma_targets.view(1, nlat, nlon) * posterior_mean
posterior_mean = prior_mean + posterior_mean

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets.view(1, nlat, nlon) * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]
wlat = torch.cos(torch.deg2rad(test_scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]

In [38]:
metrics_ssp370 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP585

In [39]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp370']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp585']}}
test_data = plaingp.make_data(test_cfg)

model = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp585/SpatialPlainGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [40]:
test_scenarios = test_data.scenarios
Xtest = test_scenarios.glob_inputs[:, 1:]
Xtest = (Xtest - model.mu) / model.sigma
test_tas = test_scenarios.tas
model = model.eval()

In [41]:
with torch.no_grad():
    posterior = model.posterior(Xtest, diag=False)
    posterior = model.likelihood(posterior)

In [42]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
ntimes = len(test_scenarios[0].timesteps)
time_slice = slice(-21, None)

prior_mean = model.mu_targets.view(1, nlat, nlon).repeat(ntimes, 1, 1)
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction =  model.sigma_targets.view(1, nlat, nlon) * posterior_mean
posterior_mean = prior_mean + posterior_mean

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets.view(1, nlat, nlon) * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]
wlat = torch.cos(torch.deg2rad(test_scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]

In [43]:
metrics_ssp585 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

In [44]:
metrics_plaingp = [metrics_ssp126, metrics_ssp245, metrics_ssp370, metrics_ssp585]
scores_plaingp = pd.DataFrame(metrics_plaingp)
scores_plaingp.index = ssps
scores_plaingp.index.name = 'SSP'

# FaIRGP

### SSP126

In [45]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp245', 'ssp370', 'ssp585']}}
train_data = fairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126']}}
test_data = fairgp.make_data(test_cfg)

model = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp126/SpatialFaIRGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [46]:
test_scenarios = test_data.scenarios
test_tas = test_scenarios.tas

model = model.eval()
with torch.no_grad():
    posterior = model.posterior(test_scenarios, diag=False)
    posterior = model.likelihood(posterior)

In [47]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
time_slice = slice(-21, None)

prior_mean = model._compute_means(test_scenarios)['ssp126']
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction = model.mu_targets + model.sigma_targets * posterior_mean
posterior_mean = prior_mean + posterior_correction

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]

In [48]:
metrics_ssp126 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP245

In [49]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp370', 'ssp585']}}
train_data = fairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp245']}}
test_data = fairgp.make_data(test_cfg)

model = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp245/SpatialFaIRGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [50]:
test_scenarios = test_data.scenarios
test_tas = test_scenarios.tas

model = model.eval()
with torch.no_grad():
    posterior = model.posterior(test_scenarios, diag=False)
    posterior = model.likelihood(posterior)

In [51]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
time_slice = slice(-21, None)

prior_mean = model._compute_means(test_scenarios)['ssp245']
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction = model.mu_targets + model.sigma_targets * posterior_mean
posterior_mean = prior_mean + posterior_correction

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]

In [52]:
metrics_ssp245 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP370

In [53]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp585']}}
train_data = fairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp370']}}
test_data = fairgp.make_data(test_cfg)

model = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp370/SpatialFaIRGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [54]:
test_scenarios = test_data.scenarios
test_tas = test_scenarios.tas

model = model.eval()
with torch.no_grad():
    posterior = model.posterior(test_scenarios, diag=False)
    posterior = model.likelihood(posterior)

In [55]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
time_slice = slice(-21, None)

prior_mean = model._compute_means(test_scenarios)['ssp370']
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction = model.mu_targets + model.sigma_targets * posterior_mean
posterior_mean = prior_mean + posterior_correction

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]

In [56]:
metrics_ssp370 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP585

In [57]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp370']}}
train_data = fairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp585']}}
test_data = fairgp.make_data(test_cfg)

model = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp585/SpatialFaIRGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [58]:
test_scenarios = test_data.scenarios
test_tas = test_scenarios.tas

model = model.eval()
with torch.no_grad():
    posterior = model.posterior(test_scenarios, diag=False)
    posterior = model.likelihood(posterior)

In [59]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
time_slice = slice(-21, None)

prior_mean = model._compute_means(test_scenarios)['ssp585']
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction = model.mu_targets + model.sigma_targets * posterior_mean
posterior_mean = prior_mean + posterior_correction

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]

In [60]:
metrics_ssp585 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

In [61]:
metrics_fairgp = [metrics_ssp126, metrics_ssp245, metrics_ssp370, metrics_ssp585]
scores_fairgp = pd.DataFrame(metrics_fairgp)
scores_fairgp.index = ssps
scores_fairgp.index.name = 'SSP'

---

In [62]:
pattern_scaling_summary = scores_pattern_scaling.aggregate(['mean', 'std'])
gp_summary = scores_plaingp.aggregate(['mean', 'std'])
fairgp_summary = scores_fairgp.aggregate(['mean', 'std'])

scores_pattern_scaling = pd.concat([scores_pattern_scaling, pattern_scaling_summary])
scores_plaingp = pd.concat([scores_plaingp, gp_summary])
scores_fairgp = pd.concat([scores_fairgp, fairgp_summary])

In [63]:
scores_fairgp['Model'] = '2FaIRGP'
scores_plaingp['Model'] = '1Plain GP'
scores_pattern_scaling['Model'] = '0FaIR'
metrics_cols = ['Bias', 'RMSE', 'MAE', 'Corr', 'LL', 'Calib95', 'CRPS']
full_scores = pd.concat([scores_fairgp, scores_plaingp, scores_pattern_scaling]).sort_index().set_index('Model', append=True)
full_scores.columns = metrics_cols
full_scores = full_scores[['RMSE', 'MAE', 'Bias', 'Corr', 'LL', 'Calib95', 'CRPS']].sort_index().round(3)

In [64]:
results = full_scores[['RMSE', 'MAE', 'Bias', 'Corr', 'LL', 'Calib95', 'CRPS']].fillna('-')
results.to_latex('./tables/ssp-experiment-spatial-scores.tex')
results

/tmp/ipykernel_37897/1249708795.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  results.to_latex('./tables/ssp-experiment-spatial-scores.tex')


RMSE    MAE   Bias   Corr      LL Calib95   CRPS
       Model                                                       
SSP126 0FaIR      0.674  0.486  0.018  0.776       -       -      -
       1Plain GP  0.682  0.494  0.102  0.777  -2.151   0.745  0.374
       2FaIRGP    0.650  0.467  0.026  0.792  -1.104   0.776  0.344
SSP245 0FaIR      0.371  0.272 -0.048  0.966       -       -      -
       1Plain GP  0.582  0.421 -0.166  0.948  -2.595   0.672  0.322
       2FaIRGP    0.359  0.264 -0.057  0.968   -0.26   0.968  0.189
SSP370 0FaIR      0.662  0.483 -0.100  0.945       -       -      -
       1Plain GP  1.161  0.823 -0.469  0.913  -3.508   0.583  0.648
       2FaIRGP    0.664  0.484 -0.033  0.944  -1.277   0.738   0.36
SSP585 0FaIR      0.810  0.597 -0.409  0.967       -       -      -
       1Plain GP  1.820  1.334 -0.291  0.688 -10.823   0.462  1.105
       2FaIRGP    0.827  0.609 -0.422  0.967   -2.03   0.619  0.469
mean   0FaIR      0.629  0.459 -0.135  0.913       -       -      -
       1Plain GP  1.061  0.768 -0.206  0.832  -4.769   0.616  0.612
       2FaIRGP    0.625  0.456 -0.122  0.918  -1.168   0.775   0.34
std    0FaIR      0.185  0.135  0.189  0.092       -       -      -
       1Plain GP  0.565  0.416  0.240  0.121   4.075   0.122  0.359
       2FaIRGP    0.195  0.143  0.203  0.084   0.727   0.145  0.115